In [1]:
!pip install datasets
import datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pickle

# File path
file_path = '/content/drive/MyDrive/1_Data/ds_clean.pkl'

# Load the dataset
with open(file_path, 'rb') as f:
    ds_clean = pickle.load(f)

# Check the dataset
print(ds_clean)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})


After a lot of research and reading I decided to use tensorflow based encoder-decoder for to build my document summarization model.

I will be following instructions for this following process.

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Step 1: Tokenize the text:**

Tokenizers for documents and summaries.

In [7]:
# Initialize tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")  # Limit to top 10,000 words

In [8]:
# Fit tokenizer on both documents and summaries of train data
all_texts = ds_clean['train']['document'] + ds_clean['train']['summary']
tokenizer.fit_on_texts(all_texts)

In [9]:
# Tokenize the documents and summaries
X_train_docs = tokenizer.texts_to_sequences(ds_clean['train']['document'])
y_train_summaries = tokenizer.texts_to_sequences(ds_clean['train']['summary'])
X_val_docs = tokenizer.texts_to_sequences(ds_clean['validation']['document'])
y_val_summaries = tokenizer.texts_to_sequences(ds_clean['validation']['summary'])

Now we can convert documents and summaries to sequence as per instructions.

We can check the firs tokenized documents and summary.

In [10]:
print("Tokenized document:", X_train_docs[0])
print("Tokenized summary:", y_train_summaries[0])

Tokenized document: [158, 1, 1099, 181, 8, 68, 335, 3294, 181, 2, 80, 224, 4, 2, 341, 49, 9, 46, 1, 900, 2, 2444, 4, 682, 431, 213, 6, 699, 431, 26, 54, 1, 10, 46, 407, 959, 5, 2483, 1439, 10, 2, 57, 3591, 772, 9, 689, 3060, 2, 1, 18, 1, 250, 52, 167, 68, 1922, 46, 4, 2, 100, 471, 1, 10, 94, 2, 1198, 1, 103, 2, 872, 341, 4329, 26, 9, 1, 52, 959, 3, 22, 894, 108, 3, 1, 1, 21, 1, 1444, 1, 1213, 3, 1, 959, 318, 7, 928, 2, 3943, 954, 11, 506, 3, 873, 1, 18, 1, 192, 18, 1, 1, 3747, 22, 1922, 2679, 1, 93, 6, 47, 2, 291, 1, 2, 1, 545, 522, 3, 1, 2939, 739, 2, 759, 1376, 6, 99, 1332, 192, 254, 331, 44, 639, 55, 3, 771, 57, 10, 303, 3315, 2, 177, 4853, 1430, 119, 1, 10, 4691, 89, 341, 5, 1, 8, 2, 57, 1198, 91, 28, 22, 1210, 1862, 420, 3, 1, 1, 524, 1492, 2, 3943, 954, 1, 27, 1620, 88, 41, 534, 1325, 508, 9, 150, 2, 1986, 220, 69, 2, 1198, 318, 1, 959, 7, 377, 131, 20, 41, 60, 1, 7, 249, 4, 2, 872, 130, 271, 2, 3943, 954, 1216, 3, 1, 206, 2, 4199, 132, 689, 984, 2, 954, 1590, 88, 2604, 51, 3930,

**Step 2: Padding and Truncating:**

Since the sequences generated by tokenizers vary in length, we need to pad them to a fixed length to esnure the same shape for training the model.

In [11]:
# Set max length for padding
max_doc_length = 100
max_summary_length = 100

In [12]:
# Pad the sequences
X_train_docs_padded = pad_sequences(X_train_docs, maxlen=max_doc_length, padding='post', truncating='post')
y_train_summaries_padded = pad_sequences(y_train_summaries, maxlen=max_summary_length, padding='post', truncating='post')
X_val_docs_padded = pad_sequences(X_val_docs, maxlen=max_doc_length, padding='post', truncating='post')
y_val_summaries_padded = pad_sequences(y_val_summaries, maxlen=max_summary_length, padding='post', truncating='post')

In [13]:
# Check the shape
print("Padded X_train shape:", X_train_docs_padded.shape)
print("Padded y_train shape:", y_train_summaries_padded.shape)

Padded X_train shape: (44972, 100)
Padded y_train shape: (44972, 100)


**Step 3: Build the Model:**

In this step, we'll create a simple encoder-decoder model using TensorFlow's Sequential API. We'll use the LSTM (Long Short-Term Memory) layers for both encoding and decoding.

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed

In [15]:
# Parameters
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
embedding_dim = 32  # Size of embedding vector
latent_dim = 32  # Latent dimensionality of the encoding space

In [16]:
# Encoder
encoder_inputs = Input(shape=(max_doc_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

In [17]:
# Decoder
decoder_inputs = Input(shape=(max_summary_length,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = TimeDistributed(Dense(vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# Model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 100, 32)        │     26,599,744 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 100, 32)        │     26,599,744 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 32), (None,    │          8,320 │ embedding[0][0]        │
│                           │ 32), (None, 32)]       │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 100, 32),      │          8,320 │ embedding_1[0][0],     │
│                           │ (None, 32), (None,     │                │ lstm[0][1], lstm[0][2] │
│                           │ 32)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, 100, 831242)    │     27,430,986 │ lstm_1[0][0]           │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 80,647,114 (307.64 MB)

 Trainable params: 80,647,114 (307.64 MB)

 Non-trainable params: 0 (0.00 B)

**Step 4: Training the Model:**

Now we will train the model, for training we will use teacher forcing algorithm which is common in sequence to sequence model tasks like summarization. We need to shift the target summaries to the right (the target sequence at time step t is the summary at time step t+1 during training).

In [20]:
import numpy as np

In [21]:
# Shift the target sequence by one timestep for teacher forcing
y_train_summaries_shifted = np.zeros_like(y_train_summaries_padded)
y_train_summaries_shifted[:, :-1] = y_train_summaries_padded[:, 1:]

In [22]:
y_val_summaries_shifted = np.zeros_like(y_val_summaries_padded)
y_val_summaries_shifted[:, :-1] = y_val_summaries_padded[:, 1:]

In [23]:
# Train the model
model.fit([X_train_docs_padded, y_train_summaries_padded],
          np.expand_dims(y_train_summaries_shifted, -1),
          epochs=5, batch_size=16, validation_data=([X_val_docs_padded, y_val_summaries_padded], np.expand_dims(y_val_summaries_shifted, -1)))

Epoch 1/5
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 1174s 414ms/step - loss: 7.1245 - val_loss: 5.9317
Epoch 2/5
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 1163s 414ms/step - loss: 5.8407 - val_loss: 5.5110
Epoch 3/5
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 1164s 414ms/step - loss: 5.4382 - val_loss: 5.2808
Epoch 4/5
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 1164s 414ms/step - loss: 5.2500 - val_loss: 5.1785
Epoch 5/5
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 1164s 414ms/step - loss: 5.1386 - val_loss: 5.0809


In [26]:
# Save the model
model.save('summary_model_v1.keras')

**Step 5: Model Evaluation:**

In [2]:
from tensorflow.keras.models import load_model

# Reload the saved model
model = load_model('/content/drive/MyDrive/1_Data/summary_model_v1.keras')

ValueError: File not found: filepath=/content/drive/MyDrive/1_Data/summary_model_v1.keras. Please ensure the file is an accessible `.keras` zip file.

Preparing the test data now.

In [28]:
# Tokenize the test data
X_test_docs = tokenizer.texts_to_sequences(ds_clean['test']['document'])
y_test_summaries = tokenizer.texts_to_sequences(ds_clean['test']['summary'])

In [29]:
# Pad the sequences for the test data
X_test_docs_padded = pad_sequences(X_test_docs, maxlen=max_doc_length, padding='post', truncating='post')
y_test_summaries_padded = pad_sequences(y_test_summaries, maxlen=max_summary_length, padding='post', truncating='post')

In [31]:
y_test_summaries_shifted = np.roll(y_test_summaries_padded, shift=1, axis=1)

In [1]:
test_loss = model.evaluate([X_test_docs_padded, y_test_summaries_padded],
                           np.expand_dims(y_test_summaries_shifted, -1))

print(f"Test Loss: {test_loss}")

NameError: name 'model' is not defined